In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import GPy

# 创建保存文件夹
output_folder = "uuv_video_prepare"
os.makedirs(output_folder, exist_ok=True)

seed = 41  # 可以是任意整数
np.random.seed(seed)

# 初始化 GMM 先验分布
means = np.array([[5, 10], [15, 5], [10, 15], [5, 5], [15, 15]])
covariances = [
    [[3, 0.5], [0.5, 3]],
    [[4, -0.5], [-0.5, 3.5]],
    [[2.5, 0.8], [0.8, 2.5]],
    [[3.5, 1], [1, 3]],
    [[3, 0], [0, 3]]
]
weights = [0.2, 0.3, 0.1, 0.25, 0.15]

# 创建网格并计算概率密度
x_grid, y_grid = np.meshgrid(np.linspace(0, 20, 50), np.linspace(0, 20, 50))
grid_points = np.column_stack([x_grid.ravel(), y_grid.ravel()])

gmm_probs = np.zeros(grid_points.shape[0])
for mean, cov, weight in zip(means, covariances, weights):
    rv = multivariate_normal(mean, cov)
    gmm_probs += weight * rv.pdf(grid_points)

# 初始化稀疏 GP
kernel = GPy.kern.RBF(input_dim=2, variance=1.0, lengthscale=1.0)
num_inducing = 30
inducing_points = grid_points[np.random.choice(grid_points.shape[0], num_inducing, replace=False)]
sparse_gp_model = GPy.models.SparseGPRegression(grid_points, gmm_probs[:, None], kernel, Z=inducing_points)
sparse_gp_model.optimize()

# 允许优化诱导点的位置
sparse_gp_model.Z.optimize = True
sparse_gp_model.optimize(messages=False, max_iters=1000)

# 初始化 UUV 和其他变量
num_uuv = 3

# 将 UUV 位置固定为左下 (2,2)，左上 (2,18)，右下 (18,2)
uuv_positions = np.array([
    [2, 2],
    [2, 18],
    [18, 2]
], dtype=float)

uuv_initial_positions = np.copy(uuv_positions)  # 复制初始状态，防止后续更新影响
uuv_status = [False] * num_uuv
uuv_targets = [None] * num_uuv
observed_points = []
observed_values = []
assigned_targets = set()  # 已分配目标点集合

# 参数设置
max_steps = 50
max_distance = 0.5  # UUV 每步最大移动距离
kappa = 1.0  # UCB 中的探索权重
target_radius = 3.0
target_position = np.array([14.0, 16.0])  # 目标实际位置
num_interest_points = 8  # 每轮兴趣点数量
min_distance_between_points = 5.0  # 兴趣点之间的最小距离

# 假设每个 UUV 初始能源不同
uuv_energy = np.array([1000.0, 800.0, 1200.0])

# 定义函数
def sense_target(current_point, target_position, target_radius):
    distance = np.linalg.norm(current_point - target_position)
    return 1 if distance <= target_radius else -0.2

def find_global_interest_points(grid_points, sparse_gp_model, kappa, num_points, min_distance):
    mu, var = sparse_gp_model.predict(grid_points)
    sigma = np.sqrt(var)
    ucb = mu.flatten() + kappa * sigma.flatten()

    # 按 UCB 从大到小排序并选择兴趣点
    sorted_indices = np.argsort(ucb)[::-1]
    selected_points = []
    selected_ucb = []
    selected_sigma = []

    for idx in sorted_indices:
        candidate_point = grid_points[idx]
        # 检查与已选点的最小距离
        if all(np.linalg.norm(candidate_point - np.array(p)) >= min_distance for p in selected_points):
            selected_points.append(candidate_point)
            selected_ucb.append(ucb[idx])
            selected_sigma.append(sigma[idx])
        if len(selected_points) == num_points:
            break

    return np.array(selected_points), np.array(selected_ucb), np.array(selected_sigma)

def auction_points(interest_points, interest_ucb, interest_sigma, uuv_positions, uuv_status, uuv_energy, assigned_targets, min_distance=3.0):
    bids = {}
    assigned_uuvs = set()
    assigned_points = set()

    for idx, point in enumerate(interest_points):
        # 检查当前兴趣点是否在已分配目标附近
        if any(np.linalg.norm(point - np.array(assigned_point)) < min_distance for assigned_point in assigned_targets):
            continue  # 跳过已分配目标附近的点

        for uuv_id, position in enumerate(uuv_positions):
            if not uuv_status[uuv_id] and uuv_energy[uuv_id] > 0:  # 只考虑未分配且有足够能源的 UUV
                distance = np.linalg.norm(position - point)
                information_gain = interest_sigma[idx]
                ucb_gain = interest_ucb[idx]

                # 能源消耗系数
                energy_cost = 0.5 * distance

                # 调整成本计算
                cost = 10 * distance - 2.0 * information_gain + 1.0 * ucb_gain + 5.0 * energy_cost

                # 确保有足够能源完成
                if cost < uuv_energy[uuv_id]:
                    if idx not in bids:
                        bids[idx] = []
                    bids[idx].append((uuv_id, cost))

    assignments = {}
    for point_idx, bid_list in bids.items():
        bid_list.sort(key=lambda x: x[1])  # 按成本排序
        for uuv_id, cost in bid_list:
            if point_idx not in assigned_points and uuv_id not in assigned_uuvs:
                assignments[point_idx] = uuv_id
                assigned_points.add(point_idx)
                assigned_uuvs.add(uuv_id)
                uuv_status[uuv_id] = True  # 标记分配任务
                assigned_targets.append(interest_points[point_idx])  # 记录已分配兴趣点

                # 更新 UUV 能源
                uuv_energy[uuv_id] -= cost
                break  # 每个兴趣点只分配给一个 UUV

    return assignments

def update_gp_with_observations(grid_points, gmm_probs, observed_points, observed_values, gp_model):
    if len(observed_points) > 0:
        observed_points_array = np.array(observed_points)
        updated_points = np.vstack([grid_points, observed_points_array])
        updated_values = np.hstack([gmm_probs, observed_values])
    else:
        updated_points = grid_points
        updated_values = gmm_probs

    gp_model.set_XY(updated_points, updated_values[:, None])
    gp_model.Z.optimize = True
    gp_model.optimize(messages=False, max_iters=1000)
    return gp_model

import matplotlib.pyplot as plt
from matplotlib import rcParams

# 设置 IEEE 风格
rcParams['font.family'] = 'Times New Roman'
rcParams['font.size'] = 14   # 字体
rcParams['axes.labelsize'] = 16
rcParams['axes.titlesize'] = 18
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['legend.fontsize'] = 14
rcParams['figure.titlesize'] = 20

# 定义 UUV 颜色和轨迹颜色
uuv_colors = ['red', 'green', 'blue']
trail_colors = ['darkred', 'darkgreen', 'darkblue']

# 记录 UUV 轨迹
positions = {f"auv{i}": [] for i in range(num_uuv)}

def save_visualization(step_count, uuv_positions, observed_points, interest_points, target_position, gp_model, grid_points, x_grid, assignments):
    fig, ax = plt.subplots(figsize=(10, 8))

    # 预测 GP 均值和方差
    mu, var = gp_model.predict(grid_points)
    gp_mu = mu.reshape(x_grid.shape)
    gp_var = var.reshape(x_grid.shape)

    # 绘制 GP 均值热图
    im = ax.imshow(gp_mu, extent=(0, 20, 0, 20), origin='lower', cmap='coolwarm', alpha=0.8)
    cbar = plt.colorbar(im, ax=ax, label='SGP Mean')
    cbar.ax.tick_params(labelsize=14)

    # 绘制观测点
    if observed_points:
        obs_points = np.array(observed_points)
        ax.scatter(obs_points[:, 0], obs_points[:, 1], c='gray', s=50, label='Observed Points',
                   edgecolor='black', linewidth=0.5)

    # 绘制兴趣点
    ax.scatter(interest_points[:, 0], interest_points[:, 1], c='purple', s=100,
               label='Interest Points', marker='x', linewidth=2)

    # 绘制目标点
    ax.scatter(target_position[0], target_position[1], marker='*', c='yellow', s=200,
               label='Target', edgecolor='black', linewidth=0.5)

    # 绘制 UUV 位置和轨迹
    for i in range(num_uuv):
        ax.scatter(uuv_positions[i, 0], uuv_positions[i, 1], c=uuv_colors[i], s=100,
                   label=f'UUV {i}', edgecolor='black', linewidth=0.5)

        # 轨迹
        if len(positions[f"auv{i}"]) > 1:
            trail = np.array(positions[f"auv{i}"])
            ax.plot(trail[:, 0], trail[:, 1], c=trail_colors[i], linestyle='-', linewidth=1.5,
                    alpha=0.7, label=f'UUV {i} Trail')

    # UUV → 兴趣点
    for point_idx, uuv_id in assignments.items():
        ax.plot([uuv_positions[uuv_id, 0], interest_points[point_idx, 0]],
                [uuv_positions[uuv_id, 1], interest_points[point_idx, 1]],
                c=uuv_colors[uuv_id], linestyle='--', linewidth=1.5, alpha=0.7,
                label=f'UUV {uuv_id} → Interest {point_idx}')

    ax.set_xlabel("X Coordinate", fontsize=16)
    ax.set_ylabel("Y Coordinate", fontsize=16)
    # ax.legend(loc='upper left', fontsize=14, framealpha=0.9)
    plt.tight_layout()

    plt.savefig(os.path.join(output_folder, f"step_{step_count:03d}.png"), dpi=300, bbox_inches='tight')
    plt.close()

# 主循环
found_target = False
assigned_targets = []
uuv_targets_per_step = []  # 用于存储每轮 UUV 的目标点

for step in range(max_steps):
    if found_target:
        print("Target found, stopping exploration.")
        break

    print(f"\nRound {step + 1}")

    # 计算兴趣点
    interest_points, interest_ucb, interest_sigma = find_global_interest_points(
        grid_points, sparse_gp_model, kappa, 
        num_points=num_interest_points, 
        min_distance=min_distance_between_points
    )
    assignments = auction_points(
        interest_points, interest_ucb, interest_sigma, 
        uuv_positions, uuv_status, 
        uuv_energy, assigned_targets, 
        min_distance=min_distance_between_points
    )

    # 记录每轮 UUV 目标点
    step_targets = {}
    for point_idx, uuv_id in assignments.items():
        step_targets[f"UUV_{uuv_id}"] = interest_points[point_idx].tolist()
        uuv_targets_per_step.append(step_targets)

    # 根据分配结果移动 UUV 并进行观测
    for point_idx, uuv_id in assignments.items():
        uuv_targets[uuv_id] = interest_points[point_idx]
        uc_value = float(interest_ucb[point_idx])
        ig_value = float(interest_sigma[point_idx])
        print(f"UUV {uuv_id} assigned to interest point {interest_points[point_idx]} "
              f"with UCB {uc_value:.2f} and Information Gain {ig_value:.2f}")

    for i in range(num_uuv):
        if uuv_targets[i] is not None:
            target = uuv_targets[i]
            distance = np.linalg.norm(uuv_positions[i] - target)

            if distance <= max_distance:
                # UUV 到达目标
                uuv_positions[i] = target
                observed_value = sense_target(target, target_position, target_radius)
                observed_points.append(target)
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {target}.")

                uuv_targets[i] = None
                uuv_status[i] = False
                print(f"UUV {i} reached point: {target} with observation value: {observed_value}")
            else:
                # UUV 还没到目标，每步只走 max_distance
                direction = (target - uuv_positions[i]) / distance
                uuv_positions[i] += direction * max_distance

                observed_value = sense_target(uuv_positions[i], target_position, target_radius)
                observed_points.append(uuv_positions[i].copy())
                observed_values.append(observed_value)

                if observed_value == 1:
                    found_target = True
                    print(f"UUV {i} found the target at {uuv_positions[i]}.")

    # 保存当前位置进轨迹
    for i in range(num_uuv):
        positions[f"auv{i}"].append(uuv_positions[i].copy())

    # 更新 GP
    sparse_gp_model = update_gp_with_observations(
        grid_points, gmm_probs, observed_points, observed_values, sparse_gp_model
    )

    # 保存可视化结果
    save_visualization(step, uuv_positions, observed_points, interest_points, 
                       target_position, sparse_gp_model, grid_points, x_grid, 
                       assignments)

print("Simulation completed.")



Round 1
UUV 2 assigned to interest point [15.10204082  4.89795918] with UCB 0.01 and Information Gain 0.00
UUV 0 assigned to interest point [4.89795918 4.89795918] with UCB 0.01 and Information Gain 0.00
UUV 1 assigned to interest point [ 4.89795918 10.20408163] with UCB 0.01 and Information Gain 0.00

Round 2

Round 3

Round 4

Round 5

Round 6

Round 7

Round 8

Round 9
UUV 0 reached point: [4.89795918 4.89795918] with observation value: -0.2
UUV 2 reached point: [15.10204082  4.89795918] with observation value: -0.2

Round 10
UUV 2 assigned to interest point [14.28571429 15.10204082] with UCB 0.03 and Information Gain 0.02
UUV 0 assigned to interest point [ 8.97959184 14.69387755] with UCB 0.02 and Information Gain 0.02

Round 11

Round 12

Round 13

Round 14

Round 15

Round 16

Round 17
UUV 1 reached point: [ 4.89795918 10.20408163] with observation value: -0.2

Round 18

Round 19

Round 20
UUV 1 assigned to interest point [ 0.         12.24489796] with UCB 0.03 and Information G

In [13]:
import json
import os

def save_simulation_results_2d(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder):
    """
    保存 2D UUV 初始位置、每步目标分配、GMM 均值和目标位置到 JSON 文件。
    """
    results = {
        "uuv_initial_positions": uuv_initial_positions.tolist(),
        "uuv_targets_per_step": uuv_targets_per_step,
        "gmm_means": means.tolist(),
        "target_position": target_position.tolist()
    }
    
    # 确保输出文件夹存在
    os.makedirs(output_folder, exist_ok=True)
    
    # 保存 JSON 文件
    with open(os.path.join(output_folder, "simulation_results_2d.json"), "w") as f:
        json.dump(results, f, indent=4)
    
    print("2D Simulation results saved to simulation_results_2d.json")

# 调用保存函数
save_simulation_results_2d(uuv_initial_positions, uuv_targets_per_step, means, target_position, output_folder)


2D Simulation results saved to simulation_results_2d.json


In [ ]:
import holoocean
import numpy as np
import json
import os
import time
import csv

# --- 1. 加载 2D 仿真结果，并修改 Z 轴 ---
file_path = "uuv_video_prepare/simulation_results_2d.json"

with open(file_path, "r") as f:
    data = json.load(f)

# 为所有坐标添加 z = -15
def add_z_coordinate(data_list, scale=10.0):
    z_values = [-15, -15, -15]  # 不同 UUV 的 Z 坐标
    return [[x * scale, y * scale, z_values[i % len(z_values)]] for i, (x, y) in enumerate(data_list)]

scale = 2.0
uuv_initial_positions = np.array(add_z_coordinate(data["uuv_initial_positions"], scale))
gmm_means = np.array(add_z_coordinate(data["gmm_means"], scale))
target_position = np.array([data["target_position"][0] * scale, data["target_position"][1] * scale, -15])

# 处理 uuv_targets_per_step
uuv_targets_per_step = []
for step_targets in data["uuv_targets_per_step"]:
    updated_targets = {key: [x* scale, y* scale, -15] for key, (x, y) in step_targets.items()}
    uuv_targets_per_step.append(updated_targets)

config = {
    "name": "TwoDUUVSimulation",
    "world": "SimpleUnderwater",
    "package_name": "Ocean",
    "main_agent": "auv0",
    "agents": [
        {
            "agent_name": f"auv{i}",
            "agent_type": "HoveringAUV",
            "sensors": [
                {
                    "sensor_type": "GPSSensor",
                    "socket": "COM",
                    "configuration": {
                        "Sigma": 0.0,
                        "Depth": 100
                    }
                }
            ],
            "control_scheme": 1,  # PD Control
            "location": uuv_initial_positions[i].tolist()
        } for i in range(3)
    ] + [
        {
            "agent_name": "torpedo",
            "agent_type": "TorpedoAUV",
            "sensors": [],
            "control_scheme": 0,  # 无需控制
            "location": target_position.tolist()
        }
    ]
}

# --- 3. 模拟参数 ---
max_steps = 5000
arrival_tolerance = 0.1
save_folder = "uuv_simulation_output23"
os.makedirs(save_folder, exist_ok=True)

# --- 4. 记录数据 ---
positions = {f"auv{i}": [] for i in range(3)}
arrival_times = {f"auv{i}": [] for i in range(3)}
last_targets = {f"auv{i}": None for i in range(3)}
previous_positions = {f"auv{i}": None for i in range(3)}

# 尾迹颜色
trail_colors = {
    "auv0": [255, 0, 0],   # 红色
    "auv1": [0, 255, 0],   # 绿色
    "auv2": [0, 0, 255]    # 蓝色
}

# --- 5. 运行 HoloOcean 仿真 ---
with holoocean.make(scenario_cfg=config) as env:
    step_counter = 0
    uuv_round_indices = {f"auv{i}": 0 for i in range(3)}
    uuv_targets_done = {f"auv{i}": False for i in range(3)}

    print("✅ HoloOcean 2D UUV 仿真开始...")

    while step_counter <= max_steps:
        step_counter += 1

        # 发送控制指令
        for i in range(3):
            auv_name = f"auv{i}"
            current_round_idx = uuv_round_indices[auv_name]

            # 如果所有轮次完成，则跳过
            if current_round_idx >= len(uuv_targets_per_step):
                uuv_targets_done[auv_name] = True
                continue

            # 获取当前 UUV 的目标
            step_targets = uuv_targets_per_step[current_round_idx]
            target = step_targets.get(f"UUV_{i}", None)

            # 发送控制指令并绘制目标点
            if target is not None:
                last_targets[auv_name] = np.array(target)
                env.draw_point(target, thickness=10, lifetime=0)
                env.act(auv_name, target)

        # 运行仿真
        states = env.tick()

        # 记录 UUV 轨迹
        for i in range(3):
            auv_name = f"auv{i}"
            current_round_idx = uuv_round_indices[auv_name]

            # 跳过已完成所有目标的 UUV
            if uuv_targets_done[auv_name]:
                continue

            try:
                current_pos = states[auv_name]["GPSSensor"][0:3]
                positions[auv_name].append(current_pos)

                # 绘制 UUV 运动轨迹
                if previous_positions[auv_name] is not None:
                    env.draw_line(
                        start=previous_positions[auv_name].tolist(),
                        end=current_pos.tolist(),
                        color=trail_colors[auv_name],
                        thickness=10.0,
                        lifetime=0
                    )
                previous_positions[auv_name] = current_pos

                # 检测是否到达目标点
                target = last_targets[auv_name]
                if np.linalg.norm(current_pos - target) <= arrival_tolerance:
                    arrival_times[auv_name].append(step_counter)
                    print(f"✅ {auv_name} 到达目标 {target}，步数: {step_counter}")

                    # 进入下一轮任务
                    uuv_round_indices[auv_name] += 1
                    if uuv_round_indices[auv_name] >= len(uuv_targets_per_step):
                        uuv_targets_done[auv_name] = True

            except KeyError as e:
                print(f"❌ 错误: {e}, 跳过 {auv_name}")

        # 如果所有 UUV 完成任务，结束仿真
        if all(uuv_targets_done.values()):
            print("🏁 所有 UUV 完成任务，仿真结束！")
            # break

    print("\n✅ 2D UUV HoloOcean 仿真完成！")



✅ HoloOcean 2D UUV 仿真开始...
✅ auv2 到达目标 [ 30.20408163   9.79591837 -15.        ]，步数: 234
✅ auv2 到达目标 [ 30.20408163   9.79591837 -15.        ]，步数: 235
✅ auv2 到达目标 [ 30.20408163   9.79591837 -15.        ]，步数: 236
✅ auv0 到达目标 [  9.79591837   9.79591837 -15.        ]，步数: 240
✅ auv0 到达目标 [  9.79591837   9.79591837 -15.        ]，步数: 241
✅ auv0 到达目标 [  9.79591837   9.79591837 -15.        ]，步数: 242
✅ auv1 到达目标 [  9.79591837  19.59183673 -15.        ]，步数: 512
✅ auv1 到达目标 [  9.79591837  19.59183673 -15.        ]，步数: 513
✅ auv1 到达目标 [  9.79591837  19.59183673 -15.        ]，步数: 514
✅ auv1 到达目标 [  9.79591837  19.59183673 -15.        ]，步数: 515
✅ auv1 到达目标 [  9.79591837  19.59183673 -15.        ]，步数: 516
✅ auv1 到达目标 [  5.71428571  14.69387755 -15.        ]，步数: 700
✅ auv2 到达目标 [ 29.3877551   30.20408163 -15.        ]，步数: 886
✅ auv2 到达目标 [ 29.3877551   30.20408163 -15.        ]，步数: 887
✅ auv2 到达目标 [ 29.3877551   30.20408163 -15.        ]，步数: 888
✅ auv2 到达目标 [ 29.3877551   30.20408163 -15.        ]，步数: 8